In [24]:
import os
from os.path import join
import nibabel as nib
from skimage.measure import label, regionprops
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [25]:
def connected_component_analysis(seg, thr=0.5):
    #print(type(seg), seg.shape, seg.min(), seg.max())
    mask = seg.copy()
    mask[mask > thr] = 255
    mask[mask <= thr] = 0

    lbl = label(mask)
    blobs = regionprops(lbl)
    #raw_bbox_list = [b.bbox for b in blobs]
    #axis_major_length_list = [b.axis_major_length for b in blobs]

    return blobs

def cystregionprops(r):
    area = r.area
    major = round(r.axis_major_length, 3)
    
    try:
        minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
    except ValueError:
        minor = np.nan

    return (area, major, minor)


In [26]:
# Folder path
#seg_folder_path = r'C:\Users\pky0507\Desktop\ilkin\pancreas_ipmn\temporary_folder_cyst_segmentations\allegheny\cyst_segmentation'
#mri_folder_path = r'C:\Users\pky0507\Desktop\ilkin\pancreas_ipmn\allegheny\reoriented'

seg_folder_path = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_cad/cyst_segmentation'
#mri_folder_path = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/mayo_florida_1/reoriented'

# List to store file paths
seg_file_paths = []
#mri_file_paths = []
ids = []

# Iterate over files in the folder
for root, dirs, files in os.walk(seg_folder_path):
    for file in files:
        print(file)
        file_path = os.path.join(root, file)
        if file.startswith(".") : continue
        seg_file_paths.append(file_path)
'''
# Print the file paths
for path in seg_file_paths:
    id = path.split("/")[-1].split(".")[:-2][-2:]
    if id==[]: continue
    id = id[-1][-2:]
    print(id)
    mri_file_paths.append(os.path.join(mri_folder_path, f"CDA-{id}.nii.gz"))
'''

CAD-41.nii.gz
CAD-176.nii.gz
CAD-39.nii.gz
CAD-68.nii.gz
CAD-81.nii.gz
CAD-53.nii.gz
CAD-60.nii.gz
CAD-217.nii.gz
CAD-133.nii.gz
CAD-55.nii.gz
CAD-202.nii.gz
CAD-128.nii.gz
CAD-113.nii.gz
CAD-28.nii.gz
CAD-189.nii.gz
CAD-107.nii.gz
CAD-110.nii.gz
CAD-84.nii.gz
CAD-31.nii.gz
CAD-175.nii.gz
CAD-130.nii.gz
CAD-36.nii.gz
CAD-62.nii.gz
CAD-63.nii.gz
CAD-120.nii.gz
CAD-33.nii.gz
CAD-34.nii.gz
CAD-134.nii.gz
CAD-119.nii.gz
CAD-102.nii.gz
CAD-14.nii.gz
CAD-44.nii.gz
CAD-162.nii.gz
CAD-135.nii.gz
CAD-246.nii.gz
CAD-03.nii.gz
CAD-88.nii.gz
CAD-147.nii.gz
CAD-291.nii.gz
CAD-40.nii.gz
CAD-297.nii.gz
CAD-96.nii.gz
CAD-197.nii.gz
CAD-251.nii.gz
CAD-245.nii.gz
CAD-58.nii.gz
CAD-127.nii.gz
CAD-137.nii.gz
CAD-93.nii.gz
CAD-49.nii.gz
CAD-95.nii.gz
CAD-43.nii.gz
CAD-86.nii.gz
CAD-262.nii.gz


'\n# Print the file paths\nfor path in seg_file_paths:\n    id = path.split("/")[-1].split(".")[:-2][-2:]\n    if id==[]: continue\n    id = id[-1][-2:]\n    print(id)\n    mri_file_paths.append(os.path.join(mri_folder_path, f"CDA-{id}.nii.gz"))\n'

In [27]:
for i in range(len(seg_file_paths)):
    print(f"pair {i+1}")
    print(seg_file_paths[i-1])

#print(len(seg_file_paths), len(mri_file_paths))

pair 1
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_cad/cyst_segmentation/CAD-262.nii.gz
pair 2
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_cad/cyst_segmentation/CAD-41.nii.gz
pair 3
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_cad/cyst_segmentation/CAD-176.nii.gz
pair 4
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_cad/cyst_segmentation/CAD-39.nii.gz
pair 5
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_cad/cyst_segmentation/CAD-68.nii.gz
pair 6
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_cad/cyst_segmentation/CAD-81.nii.gz
pair 7
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_cad/cyst_segmentation/CAD-53.nii.gz
pair 8
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_f

In [28]:

def create_df():
    column_names=['Center', 'Patient', 'Pancreas_volume_ml', 'Cyst_volume_ml', 'Panc_Cyst_volume_ml', 'Diagonal_mm', 'Panc_vol_to_Diagonal', 'Num_of_Cysts']
    # Add columns using a for loop

    # Define the number of cysts
    num_cysts = 3
    for i in range(num_cysts):
        area_column_name = f'Cyst_{i+1}_vol_ml'
        major_column_name = f'Cyst_{i+1}_major_mm'
        minor_column_name = f'Cyst_{i+1}_minor_mm'
        column_names.append(area_column_name)
        column_names.append(major_column_name)
        column_names.append(minor_column_name)

    # Create an empty DataFrame
    df = pd.DataFrame(columns = column_names)

    df.head()
    return df

In [29]:

def largest3rs(r_list, voxel_sizes):

    voxel_sizes = [voxel_sizes[2], voxel_sizes[1], voxel_sizes[0]]
    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)
    top_3 = []
    
    for r in sorted_r[:3]:
      
        cov_matrix = r.inertia_tensor

        # Perform eigendecomposition to obtain eigenvectors and eigenvalues
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

        # Determine the dimension along which the major and minor axes lie
        major_axis_dimension = np.argmax(eigenvalues)
        minor_axis_dimension = np.argmin(eigenvalues)

        # Multiply the measurements by the corresponding voxel sizes
        axis_major_length_mm = r.major_axis_length * voxel_sizes[major_axis_dimension]
        try:
            axis_minor_length_mm = r.minor_axis_length * voxel_sizes[minor_axis_dimension]
        except ValueError:
            axis_minor_length_mm = np.nan
        
        # Use the calculated measurements and axis dimensions as needed
        #print(f"Axis Major Length (mm): {axis_major_length_mm}, Dimension: {major_axis_dimension}")
        #print(f"Axis Minor Length (mm): {axis_minor_length_mm}, Dimension: {minor_axis_dimension}")

        vol_ml = r.area * voxel_sizes[0] * voxel_sizes[1] * voxel_sizes[2]/1000

        #major = round(r.axis_major_length, 3)
        
        axis_major_length_mm = round(axis_major_length_mm, 3)
        axis_minor_length_mm = round(axis_minor_length_mm, 3)
        
        '''
        try:
            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
        except ValueError:
            minor = np.nan
        '''        
        
        top_3.append((vol_ml, axis_major_length_mm, axis_minor_length_mm))
        #top_3.append((vol_ml, major, minor))

    
    return top_3


In [30]:
'''
def largest3rs(r_list):
    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)
    top_3 = []
    
    for r in sorted_r[:3]:
        area = r.area
        major = round(r.axis_major_length, 3)
        
        try:
            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
        except ValueError:
            minor = np.nan
        
        top_3.append((area, major, minor))
    
    return top_3
    '''


'\ndef largest3rs(r_list):\n    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)\n    top_3 = []\n    \n    for r in sorted_r[:3]:\n        area = r.area\n        major = round(r.axis_major_length, 3)\n        \n        try:\n            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)\n        except ValueError:\n            minor = np.nan\n        \n        top_3.append((area, major, minor))\n    \n    return top_3\n    '

In [31]:
from skimage.measure import label, regionprops
import numpy as np
def compute_diagonal_length_3D(image, voxel_size_x, voxel_size_y, voxel_size_z):
    # Ensure that image is a binary image
    #print(image.shape)
    assert np.array_equal(image, image.astype(bool)), "Input should be a binary image"

    # Label the image
    labeled_image = label(image)
    #print(np.unique(labeled_image))

    # Compute the properties of the labeled regions
    regions = regionprops(labeled_image)

    # Initialize list to store diagonal lengths
    diagonals = []
    #print("voxel",voxel_size_x, voxel_size_y, voxel_size_z)
    for region in regions:
        # Get the bounding box coordinates
        minp, minr, minc, maxp, maxr, maxc = region.bbox
        # Compute the width, height, and depth of the box
        width = (maxp - minp) * voxel_size_x
        height = (maxr - minr) * voxel_size_y
        depth = (maxc - minc) * voxel_size_z
        #print("debug--------", "depth:", (maxc - minc), "y ekseni:", (maxr - minr),  "x ekseni:", (maxp - minp))
        #print(region.bbox, width, height, depth)

        # Use Pythagorean theorem to compute the diagonal length
        diagonal = np.sqrt(width**2 + height**2 + depth**2)
        diagonals.append(diagonal)

    return diagonals

In [32]:
def get_volume_ml(label, voxel_vol_mm3):
    volume_mm3 = np.sum(label != 0) * voxel_vol_mm3
    volume_ml = volume_mm3 / 1000
    return volume_ml

In [47]:
#fig, axs = plt.subplots(1, 3, figsize=(15, 5))

df = create_df()

for eachpatientind in range(len(seg_file_paths)):
    
    pt_name = seg_file_paths[eachpatientind].split("/")[-1].split(".")[0].split("-")[1]
    print("patient",pt_name)
    #if seg_file_paths[eachpatientind].split("\\")[-1].split("_")[-1] == 'pass.nii': continue
    #if pt_name == '0089': continue
    # Read niftii image and segmentation
    #print(pt_name)
    #img_nib = nib.load(mri_file_paths[eachpatientind])
    seg_nib = nib.load(seg_file_paths[eachpatientind])
    
    #print(pt_name, img_nib.header.get_zooms(), seg_nib.header.get_zooms())
    #print(pt_name, img_nib.shape, seg_nib.shape)

    #if pt_name in ['0140', '0159', '0172']:
        #print(seg_file_paths[eachpatientind].split("\\")[-1])
        #print(pt_name, img_nib.header.get_zooms(), seg_nib.header.get_zooms())
    #else:
    #assert img_nib.header.get_zooms() == seg_nib.header.get_zooms()
    #assert img_nib.shape == seg_nib.shape


    voxel_sizes = seg_nib.header.get_zooms()
    if voxel_sizes[2]>30:
        print("!!!!",voxel_sizes)
        voxel_sizes = tuple(size / 1000 for size in voxel_sizes)
    print("fixed", voxel_sizes)
    #img = img_nib.get_fdata()
    seg = seg_nib.get_fdata()
    #print(np.unique(seg))

    #print('img', img.shape)
    #print('seg', seg.shape)

    cysts = (seg == 1)
    pancreas = (seg == 5)

    labels = sorted(np.unique(seg))
    #print("ll",labels, len(labels))
    if len(labels)==2:
        panc_label = labels[1]
        pancreas = (seg == panc_label)
        cysts = (seg == 9999)

    cysts_b = cysts.astype(int)
    pancreas_b = pancreas.astype(int)
    pancreasandcyst = np.concatenate((cysts_b, pancreas_b))

    # Find Bboxes
    blobs = connected_component_analysis(cysts, thr=0.5)
    num_of_cysts = len(blobs)

    # Calculate the voxel volume (assuming isotropic voxel size)
    voxel_vol_mm3 = voxel_sizes[0] *voxel_sizes[1] * voxel_sizes[2]  # Specify the voxel volume in your desired units
    #print(voxel_vol_mm3)
    # Calculate the volume of the segmentation mask
    cysts_volume_ml = get_volume_ml(cysts, voxel_vol_mm3)
    #print(cysts_volume_ml)
    pancreas_volume_ml = get_volume_ml(pancreas, voxel_vol_mm3)

    diagonal_lenght = max(compute_diagonal_length_3D(pancreasandcyst, voxel_sizes[0], voxel_sizes[1], voxel_sizes[2]))
    pancreas_volume_to_diagonal = pancreas_volume_ml / diagonal_lenght

    column_data = ['Mayo Florida CAD', pt_name, pancreas_volume_ml, cysts_volume_ml, pancreas_volume_ml + cysts_volume_ml, diagonal_lenght, pancreas_volume_to_diagonal, num_of_cysts]

    result = largest3rs(blobs, voxel_sizes)
    # Print the result
    for i, props in enumerate(result, start=1):
        #print(f"Top {i}: Area={props[0]}, Major={props[1]}, Minor={props[2]}")
        area, major, minor = props[0], props[1], props[2]
        column_data.append(area)
        column_data.append(major)
        column_data.append(minor)

    column_data.extend([math.nan] * (17 - len(column_data)))
    #print(column_data)

    df.loc[len(df)] = column_data

df.head()

#plt.show()

patient 41
fixed (1.0, 1.0, 1.0)
patient 176
!!!! (937.5, 937.5, 3299.9993)
fixed (0.9375, 0.9375, 3.299999267578125)
patient 39
fixed (0.566406, 0.566406, 7.700012)
patient 68
fixed (1.1875, 1.1875, 6.0)
patient 81
fixed (0.703125, 0.703125, 7.699997)
patient 53
fixed (0.703125, 0.703125, 7.700005)
patient 60
fixed (0.625, 0.625, 7.699997)
patient 217
!!!! (1171.875, 1171.875, 4400.0)
fixed (1.171875, 1.171875, 4.4)
patient 133
fixed (0.546875, 0.546875, 4.399994)
patient 55
fixed (1.0625, 1.0625, 6.0)
patient 202
!!!! (1093.75, 1093.75, 4399.9976)
fixed (1.09375, 1.09375, 4.39999755859375)
patient 128
fixed (1.0, 1.0, 1.0)
patient 113
fixed (0.546875, 0.546875, 4.400002)
patient 28
fixed (1.125, 1.125, 7.699997)
patient 189
!!!! (1093.75, 905.7617, 4399.994)
fixed (1.09375, 0.90576171875, 4.399994140625)
patient 107
fixed (0.9375, 0.9375, 4.399998)
patient 110
fixed (0.9375, 0.9375, 4.400002)
patient 84
fixed (1.0, 1.0, 7.700005)
patient 31
fixed (1.0, 1.0, 1.0)
patient 175
!!!! (937

,Center,Patient,Pancreas_volume_ml,Cyst_volume_ml,Panc_Cyst_volume_ml,Diagonal_mm,Panc_vol_to_Diagonal,Num_of_Cysts,Cyst_1_vol_ml,Cyst_1_major_mm,Cyst_1_minor_mm,Cyst_2_vol_ml,Cyst_2_major_mm,Cyst_2_minor_mm,Cyst_3_vol_ml,Cyst_3_major_mm,Cyst_3_minor_mm
0,Mayo Florida CAD,41,12.913000,0.0,12.913000,159.909349,0.080752,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mayo Florida CAD,176,43.021485,0.0,43.021485,162.354098,0.264986,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mayo Florida CAD,39,67.950141,0.0,67.950141,166.545530,0.407997,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mayo Florida CAD,68,49.403414,0.0,49.403414,209.394486,0.235935,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mayo Florida CAD,81,70.371789,0.0,70.371789,201.017604,0.350078,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# Save the DataFrame to a CSV file
df.to_csv('server_cyst_segmentation_mayo_florida_cad.csv', index=False)